In [31]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Lambda
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from numpy import array
import string

In [32]:
def load_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def process(data):
    words = data.split()
    words = [word.lower() for word in words]
    final = []
    for word in words:
        try:
            val = int(word)
        except ValueError:
            final.append(word)
    return final

In [33]:
shakespeare = load_file('data/shakespeare.txt')
data = process(shakespeare)


In [34]:
length = 9
sequences = list()
for i in range(length, len(data), 5):
    seq = data[i-length:i]
    sequences.append(seq)

In [35]:
print(sequences[0])
print(len(sequences[0]))
print(sequences[1])
print(len(data))
print(len(sequences))

['from', 'fairest', 'creatures', 'we', 'desire', 'increase,', 'that', 'thereby', "beauty's"]
9
['increase,', 'that', 'thereby', "beauty's", 'rose', 'might', 'never', 'die,', 'but']
17582
3515


In [36]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sonnet = tokenizer.texts_to_sequences(sequences)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

4290


In [37]:
sonnet = array(sonnet)
print(sonnet[1])
print(len(sonnet[1]))

[ 489    8 1437  112  720  114  133  490   14]
9


In [38]:
X = sonnet[:, :-1]
y = sonnet[:, -1]
y = to_categorical(y, num_classes=vocab_size)
print(y[1])
seq_length = X.shape[1]
print(seq_length)

[0. 0. 0. ... 0. 0. 0.]
8


In [39]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=seq_length))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Lambda(lambda x: x / 1.5))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 8, 100)            429000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
lambda_2 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4290)              433290    
Total params: 952,790
Trainable params: 952,790
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
3515/3515 [==============================] - 3s 764us/step - loss: 8.2901 - acc: 0.0165 - ETA: 3s - loss: 8.3601 - acc: 0.0113

In [41]:
seed_text = "shall i compare thee to a summer's day?\n"
seed_text = seed_text.split()
print(seed_text)
seed = []
for word in seed_text:
    seed.append(word)
print(seed)

['shall', 'i', 'compare', 'thee', 'to', 'a', "summer's", 'day?']
['shall', 'i', 'compare', 'thee', 'to', 'a', "summer's", 'day?']


In [42]:
encoded = tokenizer.texts_to_sequences([seed])[0]
encoded = np.asarray(encoded)
print(encoded)
print(encoded.shape)
print(model.predict_classes(np.reshape(encoded, (1, 8)), verbose=0))

[ 45   6 513  28   3  13 205 920]
(8,)
[36]


In [48]:
print("shall i compare thee to a summer's day?\n")
result = []
for x in range(14):
    temp_line = []
    for y in range(8):
        encoded = tokenizer.texts_to_sequences([seed])[0]
        encoded = pad_sequences([encoded], maxlen=8, truncating='pre')
        yhat = model.predict_classes(np.reshape(encoded, (1, 8)), verbose=0)
        for w, index in tokenizer.word_index.items():
            if index == yhat:
                letter = w
                temp_line.append(letter)
                seed.append(letter)
                break
    result.append(temp_line)
for line in result:
    print(' '.join(line))

shall i compare thee to a summer's day?

or than thee and despise, worst desire saying
odours immortal his to and of of of
love i she self as as as they
away lest own now own now what or
should or life, date, map too day give:
purpose sun, vulgar sight, she than then to
they you for heart showers unused black, rich
tongue black, heart heart for men's tendered store.
flesh) process simple been pen are thee, love
love eye, twire process only longer only main
took services write perjured, blame, suited, blame, servant's
star pen) corrupt power any immortal eye, unkindness
nothing annexed dead, dead, o their dream receiv'st
dear even rhyme, me, from your eyes' fever
